In [ ]:


// a few dependencies for the notebook:
import $ivy.`com.github.pathikrit::better-files:3.9.1`
import $ivy.`com.lihaoyi:ujson_2.13:1.3.8`
import $ivy.`com.lihaoyi::scalatags:0.12.0`

import better.files._
import better.files.Dsl._ 

// load historia code

val jarpath = s"/home/bounder/target/scala-2.13/soot_hopper-assembly-0.1.jar"
assert(File(jarpath).exists, "must run jupyter notebook from docker")
interp.load.cp(os.Path(jarpath))

val rerun = false

In [ ]:
import almond.interpreter.api.DisplayData
import jupyter.Displayer, jupyter.Displayers
import scala.collection.JavaConverters._

import edu.colorado.plv.bounder.ir.IntConst
import edu.colorado.plv.bounder.symbolicexecutor.state.{InitialQuery,ReceiverNonNull,PureExpr,Equals}
import edu.colorado.plv.bounder.lifestate.LifeState
import edu.colorado.plv.bounder.lifestate.LifeState.{Not,And,LSConstraint}
import edu.colorado.plv.bounder.ir.{CIEnter,CIExit,CBEnter,CBExit}

import edu.colorado.plv.bounder.{Driver,RunConfig, BounderUtil,Action} // Historia utilities
import upickle.default.read
import upickle.default.write
import edu.colorado.plv.bounder.symbolicexecutor.state.{DisallowedCallin,MemoryLeak,NamedPureVar,TopVal, InitialQueryWithStackTrace, Reachable,IntVal}
import edu.colorado.plv.bounder.lifestate.SAsyncTask
import edu.colorado.plv.bounder.PickleSpec
import edu.colorado.plv.bounder.lifestate.{AllMatchers,SJavaThreading,FragmentGetActivityNullSpec, LifeState, LifecycleSpec, RxJavaSpec, SAsyncTask, 
                                           SDialog, SpecSignatures, SpecSpace, ViewSpec}
import edu.colorado.plv.bounder.lifestate.LifeState.{AbsMsg, LSSpec, LSTrue, Signature,SubClassMatcher}
import edu.colorado.plv.bounder.symbolicexecutor.{PreciseApproxMode,LimitAppRecursionDropStatePolicy,LimitMsgCountDropStatePolicy,
                                                  DumpTraceAtLocationPolicy,LimitLocationVisitDropStatePolicy, LimitCallStringDropStatePolicy, 
                                                  LimitMaterializedFieldsDropStatePolicy}
import edu.colorado.plv.bounder.symbolicexecutor.Z3TimeoutBehavior

val v = NamedPureVar("v")
val s = NamedPureVar("s")
val m = NamedPureVar("m")
val p = NamedPureVar("p")
val a = NamedPureVar("a")
val f = NamedPureVar("f")
val l = NamedPureVar("l")
val t = NamedPureVar("t")
val r = NamedPureVar("r")
val d = NamedPureVar("d")

def getIsDocker():Boolean = {
    val res = BounderUtil.runCmdStdout("whoami")
    res.trim == "root"
}

val isDocker = getIsDocker()
// val isDocker = false //TODO: overridden

val historiaDir = if(isDocker) "/home/bounder" else "/Users/shawnmeier/Documents/source/historia/Historia/"

// define a function to call the JAR implementation of Historia with a configuration
// If changes are made to Historia, run "sbt compile" in the /home/implementation directory to regenerate the Historia JAR

def runHistoriaWithSpec(configPath:File, printThenDone:Boolean = false, outputMode:String = "MEM"):String = {
    val javaMemLimit=20 // Gb Note that this only limits JVM not JNI which can go significantly higher
    val historiaJar = jarpath
    val apkRootDir = "/Users/shawnmeier/Documents/data/historia_generalizability"
    val outDir = configPath.parent.toString
    val config = read[RunConfig](configPath.contentAsString)
    val outSubdir = config.outFolder.get.replace("${baseDirOut}",outDir)
    val cmd = s"java -Xmx${javaMemLimit}G -jar ${historiaJar} -m verify -c ${configPath} -b ${apkRootDir} -u ${outDir} -o ${outputMode} --debug"
    
    if(printThenDone){
        println(cmd)
        cmd
    }else{
        //BounderUtil.runCmdStdout(cmd, Some("/Users/shawnmeier/software/z3/build"))
        BounderUtil.runCmdFileOut(cmd, configPath.parent).toString
    }
}

// def printOutput(

def runAndPrint(configPath:File, allSpecs:Iterable[LSSpec], printThenDone:Boolean = false, outputMode:String = "MEM"):String = {
    val res = runHistoriaWithSpec(configPath,printThenDone, outputMode)
    println("run result")
    println(res)   
    println("specified messages")
    val msgSigs = allSpecs.flatMap(spec =>
                Set(spec.target) ++ spec.pred.allMsg).map(msg => msg.identitySignature)
    println(msgSigs)
    println(msgSigs.size)
    res
}


val dataDir = if(isDocker) "/home/testApps/" else "/Users/shawnmeier/Documents/data/reach_24_data/"
val notebooksDir = if(isDocker) "/home/notebooks/" else s"${historiaDir}/notebooks/"


//implement in a class so any sequence gets converted to a table that can be displayed in jupyter
trait TableAble {
  def headers:List[String]   
  def values:List[Any]
}
case class Table(v:Seq[TableAble]){

}
Displayers.register(classOf[Table], (t: Table) => {
  import scalatags.Text.all._
  Map(
    "text/html" -> {
      table(cls:="table")(
        tr(t.v.head.headers.map(v => th(v))),
        for (row <- t.v) yield tr(
          row.values.map{
              case v:String => td(v)
              case v:Int => td(v)
              case v:Float => td(v)
          }
        )
      ).render
    }
  ).asJava
})

case class CategoryRow(categoryTitle:String, shortTitle:String, subMeasures: List[(String,Any)]) extends TableAble{
    override def headers:List[String] = subMeasures.map(_._1)
    override def values:List[Any] = subMeasures.map(_._2)
}

case class RuntimeRow(dir:File, catRows:List[CategoryRow]) extends TableAble{
    def addCategories(other:RuntimeRow):RuntimeRow = {
        assert(dir == other.dir, s"cannot combine non-matching directories: ${dir} and ${other.dir}")
        RuntimeRow(dir, catRows ++ other.catRows)
    }
    def benchmarkName:String = dir.toString.split("/").last
    override def headers:List[String] = "benchmark name" :: catRows.flatMap{row => row.headers.map{hdr => s"${row.shortTitle}:${hdr}"}}
    override def values:List[Any] = benchmarkName :: catRows.flatMap{row => row.values}
}

def avg(values:Seq[Int]):Float = {
    val sum = values.sum.toFloat
    sum / values.size.toFloat
}

def countRowMatches[T](in:Seq[(T,Seq[String])], matcher:scala.util.matching.Regex):Seq[(T,Int)] = {
    in.map{case (tfile, rows) => (tfile,rows.filter(matcher.matches).size)}
}

// read and print statistics about runtime traces
// should be a directory with logcat output one trace per file named "logcat.txt", "logcat1.txt" etc
// add "WITNESS" to the end of each logcat file where the issue was reached.
def runtimeStats(dir:File):RuntimeRow = {
    val traces = dir.glob("logcat*txt")
    val traceContents = traces.map{t => (t,t.contentAsString().split("\n").toList)}.toList
    val traceCount = traceContents.size
    println(s"found ${traceCount} runtime traces")

    // compute average callbacks
    val callbackCountPerTrace = avg(countRowMatches(traceContents, ".*histInstrumentation.*cb.*".r).map(_._2))

    // compute average callins
    val callinCountPerTrace = avg(countRowMatches(traceContents, ".*histInstrumentation.*ci.*".r).map(_._2))

    // compute new messages
    val newCountPerTrace = avg(countRowMatches(traceContents, ".*histInstrumentation.*new.*".r).map(_._2))

    val witnessCount = traceContents.filter{case (tfile, rows) => rows.exists{r => r.contains("WITNESS")}}.size
    RuntimeRow(dir, List(CategoryRow("Runtime Inst", "ri", List(
        ("Witness Ratio", s"${witnessCount}/${traceCount}"),
        ("cb(n)", callbackCountPerTrace),
        ("ci(n)", callinCountPerTrace),
        ("tot", callbackCountPerTrace + callinCountPerTrace + newCountPerTrace)
    ))))
}

// compareRun is file name of matching runtime trace
def wistoriaStats(expBase:File, outputDir:File, allSpecs:Set[LSSpec], compareRun:Option[String]):RuntimeRow = {
    val result = read[Driver.LocResult]((outputDir / "result_0.txt").contentAsString)
    if(result.witnessExplanation.isEmpty){
        throw new IllegalArgumentException("Witness explanation was empty")
    }
    val wit = result.witnessExplanation.head.futureTrace.map{_.toString}
    val msgSigs = allSpecs.flatMap(spec =>
            Set(spec.target) ++ spec.pred.allMsg).map(msg => msg.identitySignature)

    val rawCount = countRowMatches(Seq(("",wit)), "CBEnter.*".r)
    val cbCount = rawCount.head._2
    val ciCount = countRowMatches(Seq(("",wit)), "CIExit.*".r).head._2
    val newCount = countRowMatches(Seq(("",wit)), "TNew.*".r).head._2

    // print trace for visual comparison (TODO: perhaps automate at some point)
    println("wistoria output:")
    wit.foreach{println}
    if(compareRun.isDefined){
        val traceContents = (expBase / compareRun.get).contentAsString.split("\n")
        val onlyTraceRows = traceContents.filter{row => row.contains("histInstrumentation")}
        val onlyTraceRowsTrimmed = onlyTraceRows.map{row => row.split("histInstrumentation")(1)}
        onlyTraceRowsTrimmed.foreach{println}
    }
    RuntimeRow(expBase, List(CategoryRow("Wistoria","wi",
        List(
            ("specified msgs(n)", msgSigs.size),
            ("realistic", if(compareRun.isDefined)"yes" else "no"),
            ("cb(n)", cbCount),
            ("ci(n)", ciCount),
            ("tot", cbCount + ciCount + newCount)
        )
    )))
}

Table(Seq(
    RuntimeRow(File("/foo/bar"), List(CategoryRow("c1","c1",List(("v1",1),("v2",2))), CategoryRow("c2","c2",List(("v3",3))))),
    RuntimeRow(File("/foo/baz"), List(CategoryRow("c1","c1",List(("v1",4),("v2",5))), CategoryRow("c2","c2",List(("v3",6))))),
))

Memory Leak
-----------

In [ ]:
// Memory leak runtime stats

val memLeakExpBase = s"${dataDir}/ChatGPT_Benchmarks/EXN_activity_leak_noStatic"
val memLeakRuntimeStats = runtimeStats(File(memLeakExpBase))
Table(Seq(memLeakRuntimeStats))

In [ ]:
// Memory Leak
object MemLeak{
    val memLeakOutputDir = File(s"/home/notebooks/reachExpGPT/MemLeak")
    val memLeakAllSpecs = Set(LifecycleSpec.Activity_onDestroy_last)
}


def runMemLeak(outputDir:File, allSpecs:Set[LSSpec]){
      val query = MemoryLeak("android.app.Activity",
        Signature("com.example.activityleak.LeakActivity$1",
        "void run()"), 39, SpecSignatures.Activity_onDestroy_exit,
        NamedPureVar("a"))

    val startMsg = AbsMsg(CIExit, SubClassMatcher("java.lang.Thread","void start\\(\\)", "Thread_start"), TopVal::t::Nil)

    // val startTrue = LSSpec(t::Nil,Nil, LSTrue, startMsg)
    val inputApk = s"${memLeakExpBase}/app_bug/app/build/outputs/apk/debug/app-debug.apk"

    val configOutputDir = File(s"${notebooksDir}/reachExpGPT/MemLeak")

    
    val cfg =  RunConfig(apkPath = inputApk.toString, 
                         timeLimit=Int.MaxValue,
                    outFolder = Some(configOutputDir.toString),
                    initialQuery = List(query), truncateOut=false,
                    specSet = PickleSpec(allSpecs, 
                                             Set(), 
                                             Set(SJavaThreading.runnableI, startMsg)
                                        ),
                    componentFilter =  None,
                    approxMode = PreciseApproxMode(true, List(LimitLocationVisitDropStatePolicy(3), 
                                                              LimitAppRecursionDropStatePolicy(3),
                                                              LimitMaterializedFieldsDropStatePolicy(Map("*" -> 2)))),
                    z3TimeoutBehavior = Some(Z3TimeoutBehavior().copy(
                            subsumeTryTimeLimit=List(200000), z3InstanceLimit=16))
                        )


    if(!outputDir.isDirectory){
        mkdir(outputDir)
    }
    val cfgPath = (outputDir / "cfg.json")
    val configCfgPath = configOutputDir / "cfg.json"
    cfgPath.overwrite(write(cfg))
    


    val justPrintCommand = false
    println(runAndPrint(cfgPath, allSpecs,justPrintCommand).replace(cfgPath.toString,configCfgPath.toString))

    if(!justPrintCommand){
        val result = read[Driver.LocResult]((outputDir / "result_0.txt").contentAsString)
        println(result.witnessExplanation)
        println(result.resultSummary)
    }
}

if(rerun){
    runMemLeak(MemLeak.memLeakOutputDir, MemLeak.memLeakAllSpecs) //uncomment to run
}


object MemLeakStats{
    val wistoria = wistoriaStats(File(memLeakExpBase),MemLeak.memLeakOutputDir, MemLeak.memLeakAllSpecs, Some("logcat.txt"))
}
Table(Seq(memLeakRuntimeStats.addCategories(MemLeakStats.wistoria)))

Bitmap Mishandling
-----------------

In [ ]:
// Bitmap mishandling runtime stats

val bitmapExpBase = s"${dataDir}/ChatGPT_Benchmarks/EXN_bitmap_mishandling"
val bitmapRuntimeStats = runtimeStats(File(bitmapExpBase))
Table(Seq(bitmapRuntimeStats))

In [ ]:
// Bitmap TODO: not quite working yet
object Bitmap{
    val setImageResource = (intval:Int) => AbsMsg(CIExit, 
                                                  SubClassMatcher("android.widget.ImageView","void setImageResource\\(int\\)","ImageView_setImageResource"), 
                                                  List[PureExpr](TopVal,TopVal,IntVal(intval)))

    //val ArrayListType = "java.util.ArrayList"
    val ArrayListType = "android.fake.IntArrayList"
    val ArrayListAddMsg = AbsMsg(CIExit, SubClassMatcher(ArrayListType, ".* add\\(.*", "ArrayList_add"), TopVal::a::v::Nil)
    val ArrayListGetMsg = AbsMsg(CIExit, SubClassMatcher(ArrayListType, ".* get\\(.*", "ArrayList_get"), v::a::TopVal::Nil)
    val getAfterAdd = LSSpec(a::v::Nil, Nil, ArrayListAddMsg, ArrayListGetMsg)

    val IntegerIntValue = AbsMsg(CIExit, SubClassMatcher("java.lang.Integer", "int intValue\\(.*", "Integer_intValue"), v::t::Nil )
    val intValEq = LSSpec(v::t::Nil, Nil, LSConstraint(v,Equals,t), IntegerIntValue)
    val BaseAdapterCallbacks = List("getCount","getItem","getItemId","getView")

    val BaseAdapter = "android.widget.BaseAdapter"
    val BaseAdapterMessages = BaseAdapterCallbacks.map{cbName => 
        (cbName,AbsMsg(CBEnter,SubClassMatcher(BaseAdapter, s".* ${cbName}\\(.*", "BaseAdapter_getView"), TopVal::a::Nil))}.toMap

    val BaseAdapter_getView_getCount = LSSpec(a::Nil, Nil, BaseAdapterMessages("getCount"), BaseAdapterMessages("getView"))

    val BaseAdapterInitCallin = AbsMsg(CIExit, SubClassMatcher(BaseAdapter, ".* <init>\\(.*", "BaseAdapter_init"), TopVal::a::Nil)

    val BaseAdapter_init_getCount = LSSpec(a::Nil, Nil, BaseAdapterInitCallin, BaseAdapterMessages("getCount"))

    val disallow = LSSpec(Nil,Nil, setImageResource(2130837508), setImageResource(2130837509).copy(mt = CIEnter))
    // val disallow = LSSpec(Nil,Nil, setImageResource(2130837508).copy(lsVars= TopVal::Nil), setImageResource(2130837509).copy(mt = CIEnter))
    val outputDir = File(s"/home/notebooks/reachExpGPT/Bitmap")
    
    
    val enableSpecs = Set[LSSpec](intValEq, getAfterAdd, BaseAdapter_getView_getCount, BaseAdapter_init_getCount)
    
    val expBase = bitmapExpBase
}


def runBitmap(outputDir:File, enableSpecs:Set[LSSpec], disallow:LSSpec){

    
    val query = DisallowedCallin("com.example.bitmapmishandle.MainActivity$ImageAdapter", 
                                   "android.view.View getView(int,android.view.View,android.view.ViewGroup)", disallow)

    // TODO: figure out why the android build changed this?
    // val inputApk = s"${Bitmap.expBase}/app_bug/app/build/intermediates/apk/debug/app-debug.apk"
    val inputApk = s"${Bitmap.expBase}/app_bug/app/build/outputs/apk/debug/app-debug.apk"

    val configOutputDir = File(s"${notebooksDir}/reachExpGPT/Bitmap")
    val allMsg = Bitmap.BaseAdapterMessages.values.toSet + Bitmap.ArrayListAddMsg + Bitmap.ArrayListGetMsg + 
        Bitmap.IntegerIntValue + SpecSignatures.Activity_init_entry + Bitmap.BaseAdapterInitCallin + SpecSignatures.Activity_onCreate_entry

    
    val cfg =  RunConfig(apkPath = inputApk.toString, 
                         timeLimit=Int.MaxValue,
                    outFolder = Some(configOutputDir.toString),
                    initialQuery = List(query), truncateOut=false,
                    specSet = PickleSpec(enableSpecs, 
                                             Set(disallow), 
                                             allMsg
                                        ),
                    componentFilter =  None,
                    approxMode = PreciseApproxMode(true, List(LimitLocationVisitDropStatePolicy(3), 
                                                              LimitAppRecursionDropStatePolicy(3),
                                                              LimitMaterializedFieldsDropStatePolicy(Map("*" -> 2)),
                                                             DumpTraceAtLocationPolicy(None, None, outputDir.toString, dumpAtCallbacks = true)
                                                             )),
                    z3TimeoutBehavior = Some(Z3TimeoutBehavior().copy(
                            subsumeTryTimeLimit=List(200000), z3InstanceLimit=16))
                        )


    if(!outputDir.isDirectory){
        mkdir(outputDir)
    }
    val cfgPath = (outputDir / "cfg.json")
    val configCfgPath = configOutputDir / "cfg.json"
    cfgPath.overwrite(write(cfg))
    


    val justPrintCommand = false
    println(runAndPrint(cfgPath, enableSpecs + disallow,justPrintCommand).replace(cfgPath.toString,configCfgPath.toString))

    if(!justPrintCommand){
        val result = read[Driver.LocResult]((outputDir / "result_0.txt").contentAsString)
        println(result.witnessExplanation)
        println(result.resultSummary)
    }
}

if(rerun){
    runBitmap(Bitmap.outputDir, Bitmap.enableSpecs, Bitmap.disallow) 
}


object BitmapStats{
    val wistoria = wistoriaStats(File(bitmapExpBase),Bitmap.outputDir, Bitmap.enableSpecs + Bitmap.disallow, Some("logcat.txt"))
}
Table(Seq(bitmapRuntimeStats.addCategories(BitmapStats.wistoria)))

Device Configuration
--------------------

(TODO: do this later because it may be harder)

Dialog Origin
-------------

In [ ]:
val dialogExpBase = s"${dataDir}/ChatGPT_Benchmarks/EXN_dialog_originally_created_here"
val dialogRuntimeStats = runtimeStats(File(dialogExpBase))
Table(Seq(dialogRuntimeStats))

Table 2: comparing wistoria and runtime
----------

In [ ]:
object DialogCreated{
    val disallow = LSSpec(a::Nil, d::Nil, SDialog.showI,SpecSignatures.Activity_onDestroy_exit)
    val enableSpecs = Set[LSSpec]()
    val outputDir = File("/home/notebooks/reachExpGPT/DialogCreated")
    val Thread_start = AbsMsg(CIExit, SubClassMatcher("java.lang.Thread", "void start\\(.*", "Thread_start"), TopVal::t::Nil)

    val DialogCallins2 = List("<init>","setMessage")
    val DialogCallins2Matcher = DialogCallins2.map{name => (name,
                                                            AbsMsg(CIExit, SubClassMatcher(SDialog.DialogC, s".* ${name}\\(.*", s"Dialog_${name}"), TopVal::d::a::Nil))}.toMap
    
    val allMsg = Set(SpecSignatures.Activity_onCreate_entry, SpecSignatures.Activity_onDestroy_entry, SJavaThreading.runnableI,Thread_start) ++ DialogCallins2Matcher.values
}
def runDialogCreated(){
    val outputDir = DialogCreated.outputDir
    val disallow = DialogCreated.disallow
    val enableSpecs = DialogCreated.enableSpecs

    
    val query = DisallowedCallin("com.example.bitmapmishandle.MainActivity", 
                                   "void onDestroy()", DialogCreated.disallow)

    // TODO: figure out why the android build changed this?
    // val inputApk = s"${Bitmap.expBase}/app_bug/app/build/intermediates/apk/debug/app-debug.apk"
    val inputApk = s"${dialogExpBase}/app_bug/app/build/outputs/apk/debug/app-debug.apk"

    val configOutputDir = File(s"${notebooksDir}/reachExpGPT/DialogCreated")
    

    
    val cfg =  RunConfig(apkPath = inputApk.toString, 
                         timeLimit=Int.MaxValue,
                    outFolder = Some(configOutputDir.toString),
                    initialQuery = List(query), truncateOut=false,
                    specSet = PickleSpec(DialogCreated.enableSpecs, 
                                             Set(DialogCreated.disallow), 
                                             DialogCreated.allMsg
                                        ),
                    componentFilter =  None,
                    approxMode = PreciseApproxMode(true, List(LimitLocationVisitDropStatePolicy(3), 
                                                              LimitAppRecursionDropStatePolicy(3),
                                                              LimitMaterializedFieldsDropStatePolicy(Map("*" -> 2)),
                                                             DumpTraceAtLocationPolicy(None, None, outputDir.toString, dumpAtCallbacks = true)
                                                             )),
                    z3TimeoutBehavior = Some(Z3TimeoutBehavior().copy(
                            subsumeTryTimeLimit=List(200000), z3InstanceLimit=16))
                        )


    if(!outputDir.isDirectory){
        mkdir(outputDir)
    }
    val cfgPath = (outputDir / "cfg.json")
    val configCfgPath = configOutputDir / "cfg.json"
    cfgPath.overwrite(write(cfg))
    


    val justPrintCommand = false
    println(runAndPrint(cfgPath, enableSpecs + disallow,justPrintCommand).replace(cfgPath.toString,configCfgPath.toString))

    if(!justPrintCommand){
        val result = read[Driver.LocResult]((outputDir / "result_0.txt").contentAsString)
        println(result.witnessExplanation)
        println(result.resultSummary)
    }
}

if(rerun){
    runDialogCreated()
}

object DialogCreatedStats{
    val wistoria = wistoriaStats(File(dialogExpBase),DialogCreated.outputDir, DialogCreated.enableSpecs + DialogCreated.disallow, Some("logcat2.txt"))
}
Table(Seq(dialogRuntimeStats.addCategories(DialogCreatedStats.wistoria)))

## Execute Twice

In [ ]:
val execTwiceBase = s"${dataDir}/ChatGPT_Benchmarks/EXN_execute_twice"
val execTwiceRuntimeStats = runtimeStats(File(execTwiceBase))
Table(Seq(execTwiceRuntimeStats))

In [ ]:
import edu.colorado.plv.bounder.lifestate.{SAsyncTask, LifeState, ViewSpec}

val task_get_status = AbsMsg(CIExit, SubClassMatcher(Set("android.os.AsyncTask"), "android.os.AsyncTask.Status getStatus\\(\\)", "taskGetStatus"), a::Nil)
val set_content_view = AbsMsg(CIExit, SubClassMatcher(Set("android.app.Activity"), "void setContentView\\(.*", "setContentView"), TopVal::v::Nil);
val find_view = AbsMsg(CIExit, SubClassMatcher(Set("android.app.Activity"), ".* findViewById\\(.*", "findViewById"),a::d::Nil)

object ExecTwice{
    val disallow = SAsyncTask.disallowDoubleExecute//LSSpec(a::Nil, d::Nil, SDialog.showI,SpecSignatures.Activity_onDestroy_exit)
    val enableSpecs = Set[LSSpec](ViewSpec.clickWhileNotDisabled)
    val outputDir = File("/home/notebooks/reachExpGPT/ExecTwice")
    // println(outputDir.exists())
   
    val allMsg = Set[LifeState.OAbsMsg](SpecSignatures.Activity_onCreate_entry, ViewSpec.onClickI, task_get_status,set_content_view, find_view)
}
def runExecTwice(){
    val outputDir = ExecTwice.outputDir
    val disallow = ExecTwice.disallow
    val enableSpecs = ExecTwice.enableSpecs

    
    val query = DisallowedCallin("com.example.bitmapmishandle.MainActivity$1", 
                                   "void onClick(android.view.View)", ExecTwice.disallow)

    val inputApk = s"${execTwiceBase}/app_bug/app/build/outputs/apk/debug/app-debug.apk"

    val configOutputDir = File(s"${notebooksDir}/reachExpGPT/ExecTwice")
    
    val cfg =  RunConfig(apkPath = inputApk.toString, 
                         timeLimit=Int.MaxValue,
                    outFolder = Some(configOutputDir.toString),
                    initialQuery = List(query), truncateOut=false,
                    specSet = PickleSpec(ExecTwice.enableSpecs, 
                                             Set(ExecTwice.disallow), 
                                             ExecTwice.allMsg
                                        ),
                    componentFilter =  None,
                    approxMode = PreciseApproxMode(true, List(LimitLocationVisitDropStatePolicy(3), 
                                                              LimitAppRecursionDropStatePolicy(3),
                                                              LimitMaterializedFieldsDropStatePolicy(Map("*" -> 2)),
                                                             DumpTraceAtLocationPolicy(None, None, outputDir.toString, dumpAtCallbacks = true)
                                                             )),
                    z3TimeoutBehavior = Some(Z3TimeoutBehavior().copy(
                            subsumeTryTimeLimit=List(200000), z3InstanceLimit=16))
                        )


    if(!outputDir.isDirectory){
        mkdir(outputDir)
    }
    val cfgPath = (outputDir / "cfg.json")
    val configCfgPath = configOutputDir / "cfg.json"
    cfgPath.overwrite(write(cfg))
    


    val justPrintCommand = false
    println(runAndPrint(cfgPath, enableSpecs + disallow,justPrintCommand).replace(cfgPath.toString,configCfgPath.toString))

    if(!justPrintCommand){
        val result = read[Driver.LocResult]((outputDir / "result_0.txt").contentAsString)
        println(result.witnessExplanation)
        println(result.resultSummary)
    }
}

if(rerun){
    runExecTwice()
}

object ExecTwiceStats{
    val wistoria = wistoriaStats(File(execTwiceBase),ExecTwice.outputDir, ExecTwice.enableSpecs + ExecTwice.disallow, Some("logcat.txt"))
}
Table(Seq(execTwiceRuntimeStats.addCategories(ExecTwiceStats.wistoria)))

## Fragment LifeCycle

In [ ]:
val fragmentLFExpBase = s"${dataDir}/ChatGPT_Benchmarks/EXN_fragment_lifecycle"
val fragmentLFRuntimeStats = runtimeStats(File(fragmentLFExpBase))
Table(Seq(fragmentLFRuntimeStats))

In [ ]:
import edu.colorado.plv.bounder.lifestate.{SAsyncTask, LifeState, ViewSpec}

    val frag_destroy_view = AbsMsg(CBEnter, SubClassMatcher(SpecSignatures.Fragment,"void onDestroyView\\(\\)", "onDestroyView"), TopVal::a::Nil);
    val frag_create_view = AbsMsg(CBEnter, SubClassMatcher(SpecSignatures.Fragment,"android.view.View onCreateView\\(.*\\)", "onCreateView"), TopVal::a::d::Nil);

    val inflate = AbsMsg(CIExit, SubClassMatcher(Set("android.view.LayoutInflater"), "android.view.View inflate\\(.*\\)", "inflate"), v::d::Nil)
    val frag_des_last =  LSSpec(a::Nil,d::Nil,frag_create_view, frag_destroy_view)

    val frag_trans = Set("androidx.fragment.app.FragmentTransaction")
    val commit_transaction = AbsMsg(CIExit,SubClassMatcher(frag_trans,"int commit\\(\\)", "commit"),TopVal::f::Nil)
    val begin_transaction = AbsMsg(CIExit,SubClassMatcher(Set("androidx.fragment.app.FragmentManager"),"androidx.fragment.app.FragmentTransaction beginTransaction\\(\\)", "beginTrans"),f::r::Nil)
    val replace = AbsMsg(CIExit, SubClassMatcher(frag_trans, "androidx.fragment.app.FragmentTransaction replace\\(.*\\)", "replace"), f::TopVal::TopVal::a::Nil)

    //Need to see the commit before onCreateView
    val commit_first = LSSpec(a::d::Nil, f::Nil, And(replace,commit_transaction), frag_create_view)
object FragmentLF{

    
    val disallow = LSSpec(a::Nil, d::Nil, SDialog.showI,SpecSignatures.Activity_onDestroy_exit)
    val enableSpecs = Set[LSSpec](frag_des_last, commit_first)
    val outputDir = File("/home/notebooks/reachExpGPT/FragmentLF")
    // println(outputDir.exists())
   
    val allMsg = Set[LifeState.OAbsMsg](frag_destroy_view, 
                                        frag_create_view,
                                        inflate,
                                        commit_transaction,
                                        replace,
                                        set_content_view,
                                        begin_transaction,
                                        SpecSignatures.Activity_onCreate_entry)
}
def runFragmentLF(){
    val outputDir = FragmentLF.outputDir
    val disallow = FragmentLF.disallow
    val enableSpecs = FragmentLF.enableSpecs

    
    val query = MemoryLeak("android.view.View", LifeState.Signature("com.example.fragmentlifecycle.ArticlesFragment", "void onDestroyView()"), 42,
                           
                           And(And(inflate,frag_destroy_view), frag_create_view),v)
    //Reachable(LifeState.Signature("com.example.fragmentlifecycle.ArticlesFragment", "void onDestroyView()"),42);

    val inputApk = s"${fragmentLFExpBase}/app_bug/app/build/outputs/apk/debug/app-debug.apk"

    val configOutputDir = File(s"${notebooksDir}/reachExpGPT/FragmentLF")
    
    val cfg =  RunConfig(apkPath = inputApk.toString, 
                         timeLimit=Int.MaxValue,
                    outFolder = Some(configOutputDir.toString),
                    initialQuery = List(query), truncateOut=false,
                    specSet = PickleSpec(FragmentLF.enableSpecs, 
                                             Set(), 
                                             FragmentLF.allMsg
                                        ),
                    componentFilter =  None,
                    approxMode = PreciseApproxMode(true, List(LimitLocationVisitDropStatePolicy(3), 
                                                              LimitAppRecursionDropStatePolicy(3),
                                                              LimitMaterializedFieldsDropStatePolicy(Map("*" -> 2)),
                                                             DumpTraceAtLocationPolicy(None, None, outputDir.toString, dumpAtCallbacks = true)
                                                             )),
                    z3TimeoutBehavior = Some(Z3TimeoutBehavior().copy(
                            subsumeTryTimeLimit=List(200000), z3InstanceLimit=8))
                        )


    if(!outputDir.isDirectory){
        mkdir(outputDir)
    }
    val cfgPath = (outputDir / "cfg.json")
    val configCfgPath = configOutputDir / "cfg.json"
    cfgPath.overwrite(write(cfg))
    
    val justPrintCommand = false
    println(runAndPrint(cfgPath, enableSpecs,justPrintCommand).replace(cfgPath.toString,configCfgPath.toString))

    if(!justPrintCommand){
        val result = read[Driver.LocResult]((outputDir / "result_0.txt").contentAsString)
        println(result.witnessExplanation)
        println(result.resultSummary)
    }
}

if(rerun){
    runFragmentLF()
}

object FragmentLFStats{
    val wistoria = wistoriaStats(File(fragmentLFExpBase),FragmentLF.outputDir, FragmentLF.enableSpecs, None)
}
//Table(Seq(fragmentLFRuntimeStats.addCategories(FragmentLFStats.wistoria)))

## Null Pointer

In [ ]:
val nullptrExpBase = s"${dataDir}/ChatGPT_Benchmarks/EXN_nullptr"
val nullptrRuntimeStats = runtimeStats(File(nullptrExpBase))
Table(Seq(nullptrRuntimeStats))

In [ ]:
import edu.colorado.plv.bounder.lifestate.{SAsyncTask, LifeState, ViewSpec}
import edu.colorado.plv.bounder.lifestate.LifeState._
import edu.colorado.plv.bounder.symbolicexecutor.state.NullVal
import edu.colorado.plv.bounder.symbolicexecutor.state._

val get_intent = AbsMsg(CIExit, SubClassMatcher(Set("android.app.Activity"), ".* getIntent\\(.*", "getIntent"),v::Nil)
val start_act = AbsMsg(CIExit,SubClassMatcher(Set("android.app.Activity"), ".* startActivity\\(.*", "startActivity"),TopVal::TopVal::v::Nil)
val get_string_extra = AbsMsg(CIExit, SubClassMatcher(Set("android.content.Intent"), ".* getStringExtra\\(.*", "getStringExtra"),a::v::d::Nil)
val put_extra = AbsMsg(CIExit, SubClassMatcher(Set("android.content.Intent"), ".* putExtra\\(.*", "putExtra"),TopVal::v::d::a::Nil)
val get_after_start = LSSpec(v::Nil, Nil, start_act,get_intent)

val internal = And(Or(put_extra, LSConstraint(a, Equals, NullVal)),
Or(Not(put_extra),LSConstraint(a, NotEquals, NullVal)))
val get_put = LSSpec(v::d::a::Nil, Nil,internal, get_string_extra)
object NullPtr{
    val disallow = LSSpec(a::Nil, d::Nil, SDialog.showI,SpecSignatures.Activity_onDestroy_exit)
    val enableSpecs = Set[LSSpec](get_after_start,get_put)
    val outputDir = File("/home/notebooks/reachExpGPT/NullPtr")
    // println(outputDir.exists())
   
    val allMsg = Set[LifeState.OAbsMsg](SpecSignatures.Activity_onCreate_entry, 
                                        SpecSignatures.Activity_onDestroy_entry, 
                                        find_view,
                                        set_content_view,
                                        get_intent,
                                        start_act,
                                        get_string_extra,
                                        put_extra
                                       )
}

def runNullPtr(){
    val outputDir = NullPtr.outputDir
    val disallow = NullPtr.disallow
    val enableSpecs = NullPtr.enableSpecs

    
    val query = ReceiverNonNull(LifeState.Signature("com.example.nullptr.SecondActivity","void onCreate(android.os.Bundle)"),25,Some(".*toUpperCase.*"));

    val inputApk = s"${nullptrExpBase}/app_bug/app/build/outputs/apk/debug/app-debug.apk"

    val configOutputDir = File(s"${notebooksDir}/reachExpGPT/NullPtr")
    
    val cfg =  RunConfig(apkPath = inputApk.toString, 
                         timeLimit=Int.MaxValue,
                    outFolder = Some(configOutputDir.toString),
                    initialQuery = List(query), truncateOut=false,
                    specSet = PickleSpec(NullPtr.enableSpecs, 
                                             Set(disallow), 
                                            NullPtr.allMsg
                                        ),
                    componentFilter =  None,
                    approxMode = PreciseApproxMode(true, List(LimitLocationVisitDropStatePolicy(3), 
                                                              LimitAppRecursionDropStatePolicy(3),
                                                              LimitMaterializedFieldsDropStatePolicy(Map("*" -> 2)),
                                                             DumpTraceAtLocationPolicy(None, None, outputDir.toString, dumpAtCallbacks = true)
                                                             )),
                    z3TimeoutBehavior = Some(Z3TimeoutBehavior().copy(
                            subsumeTryTimeLimit=List(200000), z3InstanceLimit=16))
                        )


    if(!outputDir.isDirectory){
        mkdir(outputDir)
    }
    val cfgPath = (outputDir / "cfg.json")
    val configCfgPath = configOutputDir / "cfg.json"
    cfgPath.overwrite(write(cfg))
    


    val justPrintCommand = false
    println(runAndPrint(cfgPath, enableSpecs,justPrintCommand).replace(cfgPath.toString,configCfgPath.toString))

    if(!justPrintCommand){
        val result = read[Driver.LocResult]((outputDir / "result_0.txt").contentAsString)
        println(result.witnessExplanation)
        println(result.resultSummary)
    }
}

if(rerun){
    runNullPtr()
}

object NullPtrStats{
    val wistoria = wistoriaStats(File(nullptrExpBase),NullPtr.outputDir, NullPtr.enableSpecs + NullPtr.disallow, None)
}
Table(Seq(nullptrRuntimeStats.addCategories(NullPtrStats.wistoria)))

## NotHandlingNetworkFailure

In [ ]:
val netFailBase = s"${dataDir}/ChatGPT_Benchmarks/EXN_notHandlingNetworkFailure"
val netFailRuntimeStats = runtimeStats(File(netFailBase))
Table(Seq(netFailRuntimeStats))

In [ ]:
import edu.colorado.plv.bounder.lifestate.{SAsyncTask, LifeState, ViewSpec}
import edu.colorado.plv.bounder.lifestate.LifeState._
import edu.colorado.plv.bounder.symbolicexecutor.state.NullVal
import edu.colorado.plv.bounder.symbolicexecutor.state._

object NetFail{
    val error_resp = AbsMsg(CBEnter, SubClassMatcher(Set("com.example.networkfailure.MainActivity$MyErrorListener"), "void onErrorResponse\\(.*\\)", "onErrorResponse"),
                            TopVal::f::Nil)
    val queue_add = AbsMsg(CIExit, SubClassMatcher(Set("com.android.volley.RequestQueue"), ".* add\\(.*\\)", "queueAdd"), TopVal::d::a::Nil)
    val construct_req = AbsMsg(CIExit, SubClassMatcher(Set("com.android.volley.StringRequest"), ".*<init>.*", "consRequest"),Nil)
    val construct_err = AbsMsg(CIExit, SubClassMatcher(Set("com.example.networkfailure.MainActivity$MyErrorListener"), ".*<init>.*", "consErr"), Nil)              
    val construct_with_error = LSSpec(f::Nil,Nil,construct_err,error_resp)
    val disallow = LSSpec(a::Nil, d::Nil, SDialog.showI,SpecSignatures.Activity_onDestroy_exit)
    val enableSpecs = Set[LSSpec]()
    val outputDir = File("/home/notebooks/reachExpGPT/NetFail")
    // println(outputDir.exists())

    val allMsg = Set[LifeState.OAbsMsg](error_resp,
                                        queue_add,
                                       SpecSignatures.Activity_onCreate_entry, 
                                        construct_err,
                                        construct_req)
}

def runNetFail(){
    val outputDir = NetFail.outputDir
    val disallow = NetFail.disallow
    val enableSpecs = NetFail.enableSpecs

    val query = Reachable(LifeState.Signature("com.example.networkfailure.MainActivity", "void fetchUserData()"),82)
        /*Reachable(LifeState.Signature("com.example.networkfailure.MainActivity$MyErrorListener", 
                                   "void onErrorResponse(com.android.volley.VolleyError)"), 52)*/

    val inputApk = s"${netFailBase}/app_bug/app/build/outputs/apk/debug/app-debug.apk"

    val configOutputDir = File(s"${notebooksDir}/reachExpGPT/NetFail")
    
    val cfg =  RunConfig(apkPath = inputApk.toString, 
                         timeLimit=Int.MaxValue,
                    outFolder = Some(configOutputDir.toString),
                    initialQuery = List(query), truncateOut=false,
                    specSet = PickleSpec(NetFail.enableSpecs, 
                                             Set(disallow), 
                                            NetFail.allMsg
                                        ),
                    componentFilter =  None,
                    approxMode = PreciseApproxMode(true, List(LimitLocationVisitDropStatePolicy(3), 
                                                              LimitAppRecursionDropStatePolicy(3),
                                                              LimitMaterializedFieldsDropStatePolicy(Map("*" -> 2)),
                                                             DumpTraceAtLocationPolicy(None, None, outputDir.toString, dumpAtCallbacks = true)
                                                             )),
                    z3TimeoutBehavior = Some(Z3TimeoutBehavior().copy(
                            subsumeTryTimeLimit=List(200000), z3InstanceLimit=16))
                        )


    if(!outputDir.isDirectory){
        mkdir(outputDir)
    }
    val cfgPath = (outputDir / "cfg.json")
    val configCfgPath = configOutputDir / "cfg.json"
    cfgPath.overwrite(write(cfg))
    


    val justPrintCommand = false
    println(runAndPrint(cfgPath, enableSpecs,justPrintCommand).replace(cfgPath.toString,configCfgPath.toString))

    if(!justPrintCommand){
        val result = read[Driver.LocResult]((outputDir / "result_0.txt").contentAsString)
        println(result.witnessExplanation)
        println(result.resultSummary)
    }
}

if(rerun){
    runNetFail()
}

object NetFailStats{
    val wistoria = wistoriaStats(File(netFailBase),NetFail.outputDir, NetFail.enableSpecs + NetFail.disallow, None)
}
Table(Seq(netFailRuntimeStats.addCategories(NetFailStats.wistoria)))

## Inefficient Network Call

In [ ]:
val ineffNetBase = s"${dataDir}/ChatGPT_Benchmarks/EXN_inefficient_network"
val ineffNetRuntimeStats = runtimeStats(File(ineffNetBase))
Table(Seq(ineffNetRuntimeStats))

In [ ]:
import edu.colorado.plv.bounder.lifestate.{SAsyncTask, LifeState, ViewSpec}
import edu.colorado.plv.bounder.lifestate.LifeState._
import edu.colorado.plv.bounder.symbolicexecutor.state.NullVal
import edu.colorado.plv.bounder.symbolicexecutor.state._

object IneffNet{
    // val fetchSig = SubClassMatcher(Set("com.example.inefficientnetwork.MainActivity"), ".*fetchArticles.*", "fetch_articles")
    // val fetchI = AbsMsg(CIExit, fetchSig, TopVal::t::Nil)
    // val fetchIE = AbsMsg(CIEnter,fetchSig, TopVal::t::Nil)
    val fetchSig = SubClassMatcher(Set("android.widget.TextView"), "void setText\\(java.lang.CharSequence\\)", "set_text")
    val fetchI = AbsMsg(CIExit, fetchSig, TopVal::t::TopVal::Nil)
    val fetchIE = AbsMsg(CIEnter,fetchSig, TopVal::t::TopVal::Nil)
    // val fetchSig = SubClassMatcher(Set("com.android.volley.toolbox.Volley"), ".*newRequestQueue.*", "fetch_articles")
    // val fetchI = AbsMsg(CIExit, fetchSig, TopVal::TopVal::t::Nil)
    // val fetchIE = AbsMsg(CIEnter,fetchSig, TopVal::TopVal::t::Nil)
    val disallow = LSSpec(t::Nil, Nil, fetchI, fetchIE )
    val enableSpecs = Set[LSSpec]()
    val outputDir = File("/home/notebooks/reachExpGPT/IneffNet")
    // println(outputDir.exists())

    val allMsg = Set[LifeState.OAbsMsg](
                                       SpecSignatures.Activity_onCreate_entry,
                                       fetchI,
                                       fetchIE,
        find_view,
        SpecSignatures.Activity_onResume_entry
    )
}

def runIneffNet(){
    val outputDir = IneffNet.outputDir
    val disallow = IneffNet.disallow
    val enableSpecs = IneffNet.enableSpecs

    val query = Reachable(LifeState.Signature("com.example.inefficientnetwork.MainActivity", "void fetchArticles()"),67)
    //DisallowedCallin("android.widget.TextView","void setText(java.lang.CharSequence)", IneffNet.disallow)
    //DisallowedCallin("com.example.inefficientnetwork.MainActivity", "void fetchArticles()", IneffNet.disallow)
    //DisallowedCallin("com.android.volley.toolbox.Volley", "com.android.volley.RequestQueue newRequestQueue(android.content.Context)", IneffNet.disallow)
    //Reachable(LifeState.Signature("com.example.inefficientnetwork.MainActivity", "void fetchArticles()"),104)
    val inputApk = s"${ineffNetBase}/app_bug/app/build/outputs/apk/debug/app-debug.apk"

    val configOutputDir = File(s"${notebooksDir}/reachExpGPT/IneffNet")
    
    val cfg =  RunConfig(apkPath = inputApk.toString, 
                         timeLimit=Int.MaxValue,
                    outFolder = Some(configOutputDir.toString),
                    initialQuery = List(query), truncateOut=false,
                    specSet = PickleSpec(IneffNet.enableSpecs, 
                                             Set(disallow), 
                                            IneffNet.allMsg
                                        ),
                    componentFilter =  None,
                    approxMode = PreciseApproxMode(true, List(LimitLocationVisitDropStatePolicy(3), 
                                                              LimitAppRecursionDropStatePolicy(3),
                                                              LimitMaterializedFieldsDropStatePolicy(Map("*" -> 2)),
                                                             DumpTraceAtLocationPolicy(None, None, outputDir.toString, dumpAtCallbacks = true)
                                                             )),
                    z3TimeoutBehavior = Some(Z3TimeoutBehavior().copy(
                            subsumeTryTimeLimit=List(200000), z3InstanceLimit=16))
                        )


    if(!outputDir.isDirectory){
        mkdir(outputDir)
    }
    val cfgPath = (outputDir / "cfg.json")
    val configCfgPath = configOutputDir / "cfg.json"
    cfgPath.overwrite(write(cfg))
    


    val justPrintCommand = false
    println(runAndPrint(cfgPath, enableSpecs,justPrintCommand).replace(cfgPath.toString,configCfgPath.toString))

    if(!justPrintCommand){
        val result = read[Driver.LocResult]((outputDir / "result_0.txt").contentAsString)
        println(result.witnessExplanation)
        println(result.resultSummary)
    }
}

if(true || rerun){
    runIneffNet()
}

object IneffNetStats{
    val wistoria = wistoriaStats(File(ineffNetBase),IneffNet.outputDir, IneffNet.enableSpecs + IneffNet.disallow, None)
}
Table(Seq(ineffNetRuntimeStats.addCategories(IneffNetStats.wistoria)))